<a href="https://colab.research.google.com/github/shicong621/Colab/blob/main/CS505_PA2_Student_Shicong_Wang.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Assignment 2**: (One week, due in Gradescope at midnight 9/28 with same grace period and late policy as in PS 1)

In this assignment, we are going to try building language models with 
the data we collected from various sources.

In the first half, we are going to analyze our twitter data with NLTK. 
There are several tasks we would like to you to finish during this process:

1. Preprocess the raw twitter data and make them into a format that
language models in NLTK can train with.
2. Train uni-gram, bi-gram and tri-gram models with Add-one smoothing.
3. Compute perplexity to evaluate our language models based on different test sets.
4. Generate new sentences with our language models based on the trained data.
5. Perform sentiment analysis on our scraped data.

In the following sections, we are going to provide a code template to allow you
to complete them step by step.

Here's a [general guide](https://www.kaggle.com/code/alvations/n-gram-language-model-with-nltk/notebook) of how to build language model with NLTK, please refer to this guide from time to time to see what you missed.

Please submit this code with your implementaton and outputs. **Please indicate which students, if any, you consulted with as you completed this assignment.** 

First, please go back to the code of our first lab section. Scrape 10000 tweets.
1. mentions 'fishing' / 'football'
2. is written in English
3. does not mention any other twitter account (i.e. @).
4. does not contain links.
5. is not a re-tweet.

Then, scrape 10000 tweets with the same rules above but mention 'football' instead of 'fishing' this time.

Save the scraped tweets in separate files, one for 'fishing' tweets
and one for 'football'.

## Task 1

First, let's try loading our scraped data. To begin with, let's load our 'fishing' data. You may change the following
function as necessary.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import csv

def loadTextFromCSV(csvPath):
  tweetDict = {}
  with open(csvPath, newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
      tweetDict[int(row['idx'])] = row['tweetText']
  return tweetDict

#load your fishing tweet data here:
csvPathFish = "/content/drive/MyDrive/Colab Notebooks/tweets_fishing.csv"
rawTweetDictFish = loadTextFromCSV(csvPathFish)

#print your tweet dictionary. You should see your saved tweets inside.
print("rawTweetDictFish: ",rawTweetDictFish)

rawTweetDictFish:  {0: 'fishing: 名\u3000魚釣り、釣り場、漁業権\u3000 #マイクラ\u3000#Minecraft', 1: 'Dear everyone in medical waiting rooms: your phone doesn’t HAVE to make noise. (Obligatory Twitter disclaimer: I am not talking about taking a vital call or watching something important with sound on due to limited vision. I am talking about watching fishing videos.)', 2: 'Might fuck around and get into fishing (but not in a cringe male way)', 3: 'There’s these two guys fishing at the lake at COM…why??', 4: 'kogasa   fact to day: sghigy is watching you singing a song about fishing rod', 5: 'about to take a disposable camera my dad took on a fishing trip in 2006 to get developed this should be cool!', 6: 'OH FUCK OFF of course when the fishing challenge in rdo is for largemouth bass i catch nothing but bluegills but when the challenge is for bluegills im catching nothing but bass. im literally in the exact same fishing spot too', 7: 'sometimes fishing', 8: "⠀\n\n\xa0\xa0❝\xa0 That's it Mort .\n\n It's 

In [ ]:
#load your football tweet data here:
csvPathFootball = "/content/drive/MyDrive/Colab Notebooks/tweets_football.csv"
rawTweetDictFootball = loadTextFromCSV(csvPathFootball)

#print your tweet dictionary. You should see your saved tweets inside.
print("rawTweetDictFootball: ",rawTweetDictFootball)

rawTweetDictFootball:  {0: 'Elsdon on Jack Ham: “Every time I talk with him, I talk about football with him and really talk about life with him. He’s a great person.”', 1: "Varane didn't get injured during international break? The football Gods might be on our side this season guys...", 2: 'What made you support the football team that you support now?', 3: "Watching these college football documentaries on Netflix. Do all these 300lb HC's that seem to have no personal discipline all talk to their athletes like their scum of the earth. Maybe I'm just ignorant to that culture. Seems ironic to me.", 4: 'j’aime trooooop les unpopular opinion football de Jayf2.0 et Boumé Sama sur tiktok alors que je comprends vraiment 2 opinions sur 6', 5: 'SFG getting played at football game and bands playing it… that’s a bigggg hit', 6: 'More from Bloomgren: They can put up whatever quotes they want. They’ve got to come down here to Houston, they’ve got to play these guys. I have no doubt this will be a ph

## Task 2

Next, we are going to pre-process texts with NLTK library.

Install NLTK library if it's not in your Google Colab space.

Download 'punkt' specifically for sentence segmentation.

In [ ]:
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

We preprocess our tweet data with the following steps:
1. Split data into training and testing splits. (**80%** tweets for training and **20%** tweets for testing) 
2. Sentence segmentation/spliting.
3. Lower-case all words in the sentences.
4. Tokenization (you should use TweetTokenizer from NLTK.tokenize)
5. Padding with begin-of-sentence and end-of-sentence symbols 

You may refer to the following materials:
1. [Sentence segmentation](https://www.nltk.org/api/nltk.tokenize.html). 
2. [String Lower case](https://www.w3schools.com/python/ref_string_lower.asp).
3. [Tweet Tokenization](https://www.nltk.org/api/nltk.tokenize.casual.html).
4. [Padding tokenized sentences](https://www.nltk.org/_modules/nltk/lm/preprocessing.html). Particularly, please look at function 'padded_everygram_pipeline'.

We will handle the first 3 steps in the following block.

In [ ]:
# Here's a template you may want to start with for your data pre-processing.
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.tokenize import TweetTokenizer
from sklearn.model_selection import train_test_split

def preprocess(rawTweetDataDict):
  #Input: a dictionary contains raw tweet data scraped from Tweeter
  #Output: two lists of tweet sentences (train/test), but each tweet sentence is represented in the form of tokens.
  training, testing = train_test_split(list(rawTweetDataDict.values()),train_size = 0.8)
  print('Tweets train senteces number: {}'.format(len(training)))
  print('Tweets test senteces number: {}'.format(len(testing)))
  return training, testing
  
def sentenceSegmentation(tweets):
    #Input: a string of raw tweet
    #Output: a list of strings, each element in the list is a segmented sentence
  tweet_segment = []
  for str in tweets:
    tweet = sent_tokenize(str)
    for sent in tweet:
      tweet_segment.append(sent)   
  return tweet_segment
    
def sentenceLowerCase(sentence):  
    #Input: a string of sentence
    #Output: a string of sentence, but all words in the sentence are lower-cased.
  sentence = sentence.lower()
  return sentence

def sentenceTokenization(sentence):
    #Input: a string of sentence
    #Output: a list of tokens that belong to the sentence.   
  tknzr = TweetTokenizer()
  sents = []   
  for sent in sentence:
    tokenized = tknzr.tokenize(sent)
    sents.append(tokenized)
  return sents


In [ ]:
# split data
training, testing = preprocess(rawTweetDictFish)

Tweets train senteces number: 7864
Tweets test senteces number: 1967


In [ ]:
# sentence segmentation
training = sentenceSegmentation(training)
testing = sentenceSegmentation(testing)

In [ ]:
import re
# string lower case
train_lower = []
test_lower = []

for sent in training:
  lower = sentenceLowerCase(sent)
  lower = re.sub(r'[^a-zA-Z,.?!():;\'"]', ' ', lower)
  lower = re.sub(r'\n', '', lower)
  train_lower.append(lower)

for sent in testing:
  lower = sentenceLowerCase(sent)
  lower = re.sub(r'[^a-zA-Z,.?!():;\'"]', ' ', lower)
  lower = re.sub(r'\n', '', lower)
  test_lower.append(lower)

In [ ]:
# tokenization
train_token = sentenceTokenization(train_lower)
test_token = sentenceTokenization(test_lower)

In [ ]:
# padding with begin-of-sentence and end-of-sentences symbols
from nltk.util import pad_sequence

train_data = []
test_data = []

for str in train_token:
  token = list(pad_sequence(str,
                            pad_left = True, left_pad_symbol = "<s>",
                            pad_right = True, right_pad_symbol = "</s>",
                            n = 2))
  train_data.append(token)

for str in test_token:
  token = list(pad_sequence(str,
                            pad_left = True, left_pad_symbol = "<s>",
                            pad_right = True, right_pad_symbol = "</s>",
                            n = 2))
  test_data.append(token)

## Task 3

Next, we build our n-gram model with our pre-processed data.
First we need to pad our data with padded_everygram_pipeline. Then, we train our n-gram model with add-one smoothing using the corresponding functions in NLTK.

Related materials:
1. [Padding](https://www.nltk.org/_modules/nltk/lm/preprocessing.html)
2. [N-gram Language Model](https://www.nltk.org/api/nltk.lm.html)

Let's train unigram, bigram and trigram models with our train data split.

In [ ]:
from nltk.lm.models import Laplace
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.lm.preprocessing import pad_both_ends
from nltk.lm.preprocessing import flatten

In [ ]:
# Here's a template you may want to start with
def trainNGramAddOneSmoothing(trainData,N_GRAM):
  # Input: a list of tweet sentences, each element is a list of tokens; n for ngram model
  # Output: a n-gram model with add-one smoothing trained on your input data. 
  train, vocab = padded_everygram_pipeline(N_GRAM, trainData) # n-number for n-gram
  model = Laplace(N_GRAM) # Lets train a n-grams model, previously we set n=1,2,3
  model.fit(train, vocab)
  print("Ngram model: {}".format(model.vocab))
  return model
  

In [ ]:
UNI_GRM = 1
BI_GRM = 2
TRI_GRM = 3
uni_model = trainNGramAddOneSmoothing(train_data, UNI_GRM)
bi_model = trainNGramAddOneSmoothing(train_data, BI_GRM)
tri_model = trainNGramAddOneSmoothing(train_data, TRI_GRM)

Ngram model: <Vocabulary with cutoff=1 unk_label='<UNK>' and 16976 items>
Ngram model: <Vocabulary with cutoff=1 unk_label='<UNK>' and 16976 items>
Ngram model: <Vocabulary with cutoff=1 unk_label='<UNK>' and 16976 items>


## Task 4

Now we apply our analysis on the trained model. 

First, compute the average perplexity of your tri-gram model on the sentences of our test data.

[How to compute perplexity](https://www.nltk.org/api/nltk.lm.html)

Next, load the tweet data of 'football' instead, and compute the perplexity of your 'fishing' model on the football tweets. 

**Why is there a difference between the two perplexities, what causes it?**
(Please answer in a text cell.)

In [ ]:
# Here's a template you may want to start with
from nltk.lm import MLE
import numpy as np
def computePerplexity(model, N_GRAM, testData):
  # Input: your model; the testing data
  # Output: average perplexity of the model on your testing data.
  test_data, padded_sents_test = padded_everygram_pipeline(N_GRAM, testData)
  ngram_perplex = []
  n = 0
  for ngramlize_sent in test_data:
    ngram_list = list(ngramlize_sent)
    ngram_sent = []
    for ngram in ngram_list:
      if len(ngram) == N_GRAM:
        ngram_sent.append(ngram)
        ngram_perplex.append(model.perplexity(ngram_sent))

  perplex = np.mean(ngram_perplex)
  return perplex
  
# compute 
# 1. the perplexity of your model on your testing data of 'fishing' tweets.
# 2. the perplexity of your model on your data of 'football' tweets.

In [ ]:
# 1. the perplexity of your model on your testing data of 'fishing' tweets.
tri_perplex1 = computePerplexity(tri_model, TRI_GRM, train_data)
print("Trigram perplex of testing data of 'fishing': {}".format(tri_perplex1))

Trigram perplex of testing data of 'fishing': 2123.874646703553


In [ ]:
# load the football data
football = list(rawTweetDictFootball.values())
football = sentenceSegmentation(football)

football_lower = []
for sent in football:
  lower = sentenceLowerCase(sent)
  lower = re.sub(r'[^a-zA-Z,.?!():;\'"]', ' ', lower)
  lower = re.sub(r'\n', '', lower)
  football_lower.append(lower)

football_token = sentenceTokenization(football_lower)


In [ ]:
football_data = []
for str in football_token:
  token = list(pad_sequence(str,
                            pad_left = True, left_pad_symbol = "<s>",
                            pad_right = True, right_pad_symbol = "</s>",
                            n = 2))
  football_data.append(token)

In [ ]:
# 2. the perplexity of your model on your data of 'football' tweets.
tri_perplex2 = computePerplexity(tri_model, TRI_GRM, football_data)
print("Trigram perplex of 'football' tweets: {}".format(tri_perplex2))

Trigram perplex of 'football' tweets: 4698.315912585813


#### Difference and Reason:
The preplexities of two different data sets are related to what strings does the model cover, or what is called a “closed vocabulary”. Apparently, the test data set of fishing data is more related to the training data set of fishing, whereas the football data is much less related.

## Task 5

Next, generate 10 tweets using each of your language models (unigram, bigram, trigram). The generated tweets needs to be in string format instead of tokens, also the string should be without padding.

[Generate new sentences with your model.](https://www.nltk.org/api/nltk.lm.html)

[Detokenize your generated sentences](https://www.nltk.org/howto/tokenize.html)

In [ ]:
from nltk.lm.models import MLE
from nltk.tokenize.treebank import TreebankWordDetokenizer

In [ ]:
def train_lm_model_for_generate(N_GRAM, train_sentences):
    train_data, padded_sents = padded_everygram_pipeline(N_GRAM, train_sentences)
    model = MLE(N_GRAM) # Lets train a n-grams model, previously we set n=1,2,3
    model.fit(train_data, padded_sents)
    print("Ngram model: {}".format(model.vocab))
    return model

In [ ]:
uni_model = train_lm_model_for_generate(UNI_GRM, train_data)
bi_model = train_lm_model_for_generate(BI_GRM, train_data)
tri_model = train_lm_model_for_generate(TRI_GRM, train_data)

Ngram model: <Vocabulary with cutoff=1 unk_label='<UNK>' and 16976 items>
Ngram model: <Vocabulary with cutoff=1 unk_label='<UNK>' and 16976 items>
Ngram model: <Vocabulary with cutoff=1 unk_label='<UNK>' and 16976 items>


In [ ]:
# Here's a template you may want to start with

def generateNewSentence(model, num_words,randomSeed = 42):
  # Input: your model; random seed that get you different generated sentence
  # Output: a new sentence generated by your model, but in a string format instead of tokens.
  detokenize = TreebankWordDetokenizer().detokenize
  content = []
  for token in model.generate(num_words):
    if token == '<s>':
      continue
    if token == '</s>':
      break
    content.append(token)
  return detokenize(content) 

# Make loops to generate 10 tweets for each of your model (unigram, bigram and trigram)

In [ ]:
unigram_generate_tweets = []
bigram_generate_tweets = []
trigram_generate_tweets = []

for i in range(10):
    unigram_generate_tweets.append(generateNewSentence(uni_model, 20))
    bigram_generate_tweets.append(generateNewSentence(bi_model, 20))
    trigram_generate_tweets.append(generateNewSentence(tri_model, 20))

In [ ]:
unigram_generate_tweets[:5]

['',
 "follow of is you doug fishing a still i'm the the was",
 "and, work you're i sing or have that's the .. fishing pag fish ka in",
 '',
 'to for']

In [ ]:
bigram_generate_tweets[:5]

['duu',
 'committed to heart felt more fishing ask fisherman.',
 'end, the rocks at work when you...or " lets be normal skin brought.',
 ', mexicans work today.',
 'all.']

In [ ]:
trigram_generate_tweets[:5]

[':',
 'compliments bc i didnt know genshin had so much better.',
 'just a touch?',
 "manages it don't be shocked if you want happiness for a day, go offshore fishing, you can change",
 "to mock anyone and i know you'll beat me in the way it s either going to showcases."]

## Task 6

Lastly, we want to perform sentiment analysis on our collected data.
This time we will use VADER.

Please check out the following material:

[Sentiment analysis with VADER](https://www.geeksforgeeks.org/python-sentiment-analysis-using-vader/)

Then do the following:

1. Compute the ratios of positive and negative sentences in your collected data.
2. Compute the average compound sentiment of the tweets for 'fishing' and 'football'. Are they generally positive or negative? (Answer in a text cell.)
3. Compute the top 10 non stop words from positive tweets of 'fishing'. Please check out [here](https://www.geeksforgeeks.org/removing-stop-words-nltk-python/) to find out how to remove stop words in your sentences. The top 10 words shall also not include puncutations, including symbols like parenthesis ' ", or ...
You can refer to [here](https://docs.python.org/3/library/string.html) to see how to exclude them (still there will be special cases, please remember to remove them as well.)

In [ ]:
# install VADER
!pip install vaderSentiment

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 125 kB 2.2 MB/s 


### sentiment analysis

In [ ]:
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
senti_analyzer = SentimentIntensityAnalyzer()

In [ ]:
fishing = list(rawTweetDictFish.values())

In [ ]:
# Input: a list of sentences from tweets
# Output: a list of sentences from positive tweets, average compound from all the input sentences

def computeSentimentOfSentences(sentenceData):
  compound_score = []
  num_postive = 0
  num_neutral = 0
  num_negative = 0
  pos_twitter = []
  neu_twitter = []
  neg_twitter = []
  for text in sentenceData:
    sentiment = senti_analyzer.polarity_scores(text)
    score = sentiment['compound']
    compound_score.append(score)
    if score >= 0.05:
      num_postive += 1
      pos_twitter.append(text)
    elif score < 0.05 and score > -0.05: 
      num_neutral += 1
      neu_twitter.append(text)
    elif score <= -0.05:
      num_negative += 1
      neg_twitter.append(text)
    else:
      print("no match!!")
  pos_ratio = round(num_postive / (num_postive + num_neutral + num_negative), 2)
  neg_ratio = round(num_negative / (num_postive + num_neutral + num_negative), 2)
  return pos_twitter
  print("pos: {}  neu: {}  neg: {}".format(num_postive,num_neutral,num_negative))
  print("Average compound score: {}".format(np.mean(compound_score)))
  print("positive ratio: {}  negtive ratio: {}".format(pos_ratio, neg_ratio))

In [ ]:
computeSentimentOfSentences(fishing)

pos: 4776  neu: 8920  neg: 2948
Average compound score: 0.06316751982696468
positive ratio: 0.29  negtive ratio: 0.18


In [ ]:
computeSentimentOfSentences(football)

pos: 6108  neu: 7852  neg: 3823
Average compound score: 0.07193431367035934
positive ratio: 0.34  negtive ratio: 0.21


For 'fishing' and 'football' data, they are both generally positive.

### top 10 non stop words from positive tweets of 'fishing'

In [ ]:
from collections import Counter
import string
import re

In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
import nltk
# need to download 'stopwords' before using it.
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

def removeStopWords(tweet):
  lemmatizer = WordNetLemmatizer()  
  stop_words = set(stopwords.words('english'))
  tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    # remove emoji
  tweet = re.sub(r'[\U00010000-\U0010ffff]', '', tweet)
    # remove hashtags
    # only removing the hash # sign from the word
  tweet = re.sub(r'#', '', tweet)
    # remove stock market tickers like $GE
  tweet = re.sub(r'\$\w*', '', tweet)
    # remove old style retweet text "RT"
  tweet = re.sub(r'^RT[\s]+', '', tweet)
    # tokenize tweets
  tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)
  tweet_tokens = tokenizer.tokenize(tweet)

  tweets_clean = []
  for word in tweet_tokens:
        # remove stopwords, remove punctuation
      if word not in stop_words and word not in string.punctuation:
          if len(word) > 1 and re.search('[a-zA-Z]',word):
              lem_word = lemmatizer.lemmatize(word) 
              tweets_clean.append(lem_word)

  return tweets_clean

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
pos_fish = computeSentimentOfSentences(fishing)

In [ ]:
pos_words = []
for tweet in pos_fish:
    tweet_clean = removeStopWords(tweet)
    pos_words.extend(tweet_clean)
pos_counts = Counter(pos_words)

In [ ]:
pos_counts.most_common(10)

[('fishing', 3744),
 ('like', 769),
 ('go', 471),
 ('fish', 367),
 ('good', 324),
 ('want', 315),
 ('love', 300),
 ('day', 295),
 ('one', 261),
 ('time', 238)]